# Imports

In [1]:
from copy import deepcopy
from functools import partial
from pprint import pprint
import os.path as osp

In [2]:
import sacred
import torch
import torch.nn as nn
from sacred import SETTINGS
from sacred.utils import apply_backspaces_and_linefeeds
from torch.backends import cudnn

In [3]:
from torch.utils.data import DataLoader, RandomSampler, BatchSampler
from typing import NamedTuple, Optional

In [4]:
from models.matcher import MatchERT
from sacred import Ingredient

In [5]:
from models.ingredient import model_ingredient, get_model
from utils import pickle_load
from utils.data.dataset_ingredient import data_ingredient, get_loaders
from utils.data.dataset import FeatureDataset
# from utils.training import evaluate_time as evaluate
from utils.training import evaluate
ex = sacred.Experiment('RRT Evaluation', ingredients=[data_ingredient, model_ingredient], interactive=True)

In [6]:
# Filter backspaces and linefeeds
SETTINGS.CAPTURE_MODE = 'sys'
ex.captured_out_filter = apply_backspaces_and_linefeeds

In [7]:
cpu = False  # Force training on CPU
cudnn_flag = 'benchmark'
temp_dir = osp.join('logs', 'temp')
resume = None
resume = '/mnt/beegfs/home/smessoud/RerankingTransformer/RRT_GLD/rrt_gld_ckpts/r50_gldv1.pt'
seed = 0

In [8]:
device = torch.device('cuda:0' if torch.cuda.is_available() and not cpu else 'cpu')
torch.manual_seed(seed)

In [9]:
def read_file(filename):
    with open(filename) as f:
        lines = f.read().splitlines()
    return lines

In [10]:
def get_sets(desc_name, 
        train_data_dir, test_data_dir, 
        train_txt, test_txt, test_gnd_file, 
        max_sequence_len=500):
    ####################################################################################################################################
    train_lines     = read_file(osp.join(train_data_dir, train_txt))
    train_samples   = [(line.split(',')[0], int(line.split(',')[1]), int(line.split(',')[2]), int(line.split(',')[3])) for line in train_lines]
    train_set       = FeatureDataset(train_data_dir, train_samples, desc_name, max_sequence_len)
    query_train_set = FeatureDataset(train_data_dir, train_samples, desc_name, max_sequence_len)
    ####################################################################################################################################
    test_gnd_data = None if test_gnd_file is None else pickle_load(osp.join(test_data_dir, test_gnd_file))
    query_lines   = read_file(osp.join(test_data_dir, test_txt[0]))
    gallery_lines = read_file(osp.join(test_data_dir, test_txt[1]))
    query_samples   = [(line.split(',')[0], int(line.split(',')[1]), int(line.split(',')[2]), int(line.split(',')[3])) for line in query_lines]
    gallery_samples = [(line.split(',')[0], int(line.split(',')[1]), int(line.split(',')[2]), int(line.split(',')[3])) for line in gallery_lines]
    gallery_set = FeatureDataset(test_data_dir, gallery_samples, desc_name, max_sequence_len)
    query_set   = FeatureDataset(test_data_dir, query_samples,   desc_name, max_sequence_len, gnd_data=test_gnd_data)
        
    return (train_set, query_train_set), (query_set, gallery_set)

In [11]:
class MetricLoaders(NamedTuple):
    train: DataLoader
    num_classes: int
    query: DataLoader
    query_train: DataLoader
    gallery: Optional[DataLoader] = None

In [12]:
def get_loaders(desc_name, train_data_dir, 
    batch_size=36, test_batch_size=36, 
    num_workers=8, pin_memory=True, 
    sampler='random', recalls=[1, 5, 10],
    num_candidates=100):

    (train_set, query_train_set), (query_set, gallery_set) = get_sets(desc_name, 
        train_data_dir=train_data_dir,
        test_data_dir=train_data_dir,
        train_txt='test_query.txt',
        test_txt=('test_query.txt', 'test_gallery.txt'),
        test_gnd_file='gnd_roxford5k.pkl', 
        max_sequence_len=500)

    if sampler == 'random':
        train_sampler = BatchSampler(RandomSampler(train_set), batch_size=batch_size, drop_last=False)
    elif sampler == 'triplet':
        train_nn_inds = osp.join(train_data_dir, 'nn_inds_%s.pkl'%desc_name)
        train_sampler = TripletSampler(train_set.targets, batch_size, train_nn_inds, num_candidates)
    else:
        raise ValueError('Invalid choice of sampler ({}).'.format(sampler))
    train_loader = DataLoader(train_set, batch_sampler=train_sampler, num_workers=num_workers, pin_memory=pin_memory)
    query_train_loader = DataLoader(query_train_set, batch_size=test_batch_size, num_workers=num_workers, pin_memory=pin_memory)
        
    query_loader   = DataLoader(query_set, batch_size=test_batch_size, num_workers=num_workers, pin_memory=pin_memory)
    gallery_loader = DataLoader(gallery_set, batch_size=test_batch_size, num_workers=num_workers, pin_memory=pin_memory)

    return MetricLoaders(train=train_loader, query_train=query_train_loader, query=query_loader, gallery=gallery_loader, num_classes=len(train_set.categories)), recalls

In [13]:
(train_set, query_train_set), (query_set, gallery_set) = get_sets('r50_gldv1',
            '/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/data/oxford5k',
            '/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/data/oxford5k',
            'test_query.txt',
            ('test_query.txt', 'test_gallery.txt'),
            'gnd_roxford5k.pkl',
            500)

In [14]:
batch_size      = 16
test_batch_size = 16
max_sequence_len = 500
sampler = 'random'
if sampler == 'random':
   train_sampler = BatchSampler(RandomSampler(train_set), batch_size=batch_size, drop_last=False)

In [15]:
num_workers = 8  # number of workers used ot load the data
pin_memory  = True  # use the pin_memory option of DataLoader 
num_candidates = 100
recalls = [1, 5, 10]

In [16]:
train_loader = DataLoader(train_set, batch_sampler=train_sampler, num_workers=num_workers, pin_memory=pin_memory)
query_train_loader = DataLoader(query_train_set, batch_size=test_batch_size, num_workers=num_workers, pin_memory=pin_memory)

In [17]:
query_loader   = DataLoader(query_set, batch_size=test_batch_size, num_workers=num_workers, pin_memory=pin_memory)
gallery_loader = DataLoader(gallery_set, batch_size=test_batch_size, num_workers=num_workers, pin_memory=pin_memory)

In [18]:
#loaders, recall_ks = MetricLoaders(train=train_loader, query_train=query_train_loader, query=query_loader, gallery=gallery_loader, num_classes=len(train_set.categories)), recalls

In [19]:
loaders, recall_ks = get_loaders(desc_name='r50_gldv1',
    train_data_dir='/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/data/oxford5k', 
    batch_size=36, test_batch_size=36, 
    num_workers=8, pin_memory=True, 
    sampler='random', recalls=[1, 5, 10],
    num_candidates=100)

In [21]:
model_ingredient = Ingredient('model', interactive=True)

In [22]:
def get_model(num_global_features, num_local_features, seq_len, dim_K, dim_feedforward, nhead, num_encoder_layers, dropout, activation, normalize_before):
    return MatchERT(d_global=num_global_features, d_model=num_local_features, seq_len=seq_len, d_K=dim_K, nhead=nhead, num_encoder_layers=num_encoder_layers, 
            dim_feedforward=dim_feedforward, dropout=dropout, activation=activation, normalize_before=normalize_before)

In [23]:
name = 'rrt'
num_global_features = 2048  
num_local_features = 128  
seq_len = 1004
dim_K = 256
dim_feedforward = 1024
nhead = 4
num_encoder_layers = 6
dropout = 0.0 
activation = "relu"
normalize_before = False

In [24]:
model = get_model(num_global_features,num_local_features,seq_len,dim_K,dim_feedforward,nhead,num_encoder_layers,dropout,activation,normalize_before)

In [25]:
if resume is not None:
   checkpoint = torch.load(resume, map_location=torch.device('cpu'))
   model.load_state_dict(checkpoint['state'], strict=True)

In [26]:
model.to(device)
model.eval()

MatchERT(
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=1024, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
        (linear2): Linear(in_features=1024, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.0, inplace=False)
        (dropout2): Dropout(p=0.0, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=1024, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
        (linear2): Lin

In [27]:
loaders.query.dataset.desc_name, loaders.query.dataset.desc_name, loaders.query.dataset.data_dir

('r50_gldv1',
 'r50_gldv1',
 '/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/data/oxford5k')

In [31]:
nn_inds_path = osp.join(loaders.query.dataset.data_dir, 'nn_inds_%s.pkl'%loaders.query.dataset.desc_name)
cache_nn_inds = torch.from_numpy(pickle_load(nn_inds_path)).long()

In [34]:
cache_nn_inds.size()

torch.Size([70, 4993])

In [33]:
num_samples, top_k = cache_nn_inds.size()
top_k = min(top_k, 100)
top_k

100

In [28]:
eval_function = partial(evaluate, model=model, 
        cache_nn_inds=cache_nn_inds,
        recall=recall_ks, query_loader=loaders.query, gallery_loader=loaders.gallery)

In [29]:
metrics = eval_function()

  0%|          | 0/100 [00:00<?, ?it/s]

>> revisited: mAP E: 91.32, M: 75.53, H: 54.51
>> revisited: mP@k[1, 5, 10] E: [98.53 96.27 94.89], M: [97.14 94.81 92.24], H: [92.86 85.14 75.71]


100%|██████████| 100/100 [00:16<00:00,  6.00it/s]


>> revisited: mAP E: 0.46, M: 29.21, H: 56.35
>> revisited: mP@k[1, 5, 10] E: [0. 0. 0.], M: [92.86 74.71 64.43], H: [92.86 85.43 76.57]


In [30]:
pprint(metrics)

{'H_map': 56.35,
 'H_mp': [92.86, 85.43, 76.57],
 'M_map': 75.53,
 'M_mp': [97.14, 94.81, 92.24]}


In [31]:
best_val = (0, metrics, deepcopy(model.state_dict()))

In [32]:
best_val[1]

{'M_map': 75.53,
 'H_map': 56.35,
 'M_mp': [97.14, 94.81, 92.24],
 'H_mp': [92.86, 85.43, 76.57]}

In [33]:
train_loader = DataLoader(train_set, batch_sampler=train_sampler, num_workers=num_workers, pin_memory=pin_memory)
query_train_loader = DataLoader(query_train_set, batch_size=test_batch_size, num_workers=num_workers, pin_memory=pin_memory)
query_loader   = DataLoader(query_set, batch_size=test_batch_size, num_workers=num_workers, pin_memory=pin_memory)
gallery_loader = DataLoader(gallery_set, batch_size=test_batch_size, num_workers=num_workers, pin_memory=pin_memory)

In [34]:
from tqdm import tqdm

In [35]:
torch.cuda.empty_cache()

In [36]:
device = next(model.parameters()).device                                                                                         
to_device = lambda x: x.to(device, non_blocking=True)                                                                            
query_global, query_local, query_mask, query_scales, query_positions, query_names = [], [], [], [], [], []                 
gallery_global, gallery_local, gallery_mask, gallery_scales, gallery_positions, gallery_names = [], [], [], [], [], []

In [37]:
for entry in tqdm(query_loader, desc='Extracting query features', leave=False, ncols=80): 
     q_global, q_local, q_mask, q_scales, q_positions, _, q_names = entry 
     query_global.append(q_global.cpu()) 
     query_local.append(q_local.cpu()) 
     query_mask.append(q_mask.cpu()) 
     query_scales.append(q_scales.cpu()) 
     query_positions.append(q_positions.cpu()) 
     query_names.extend(list(q_names))

In [38]:
query_global    = torch.cat(query_global, 0)
query_local     = torch.cat(query_local, 0)
query_mask      = torch.cat(query_mask, 0)
query_scales    = torch.cat(query_scales, 0)
query_positions = torch.cat(query_positions, 0)

In [39]:
for entry in tqdm(gallery_loader, desc='Extracting gallery features', leave=False, ncols=80):
    g_global, g_local, g_mask, g_scales, g_positions, _, g_names = entry
    gallery_global.append(g_global.cpu())
    gallery_local.append(g_local.cpu())
    gallery_mask.append(g_mask.cpu())
    gallery_scales.append(g_scales.cpu())
    gallery_positions.append(g_positions.cpu())
    gallery_names.extend(list(g_names))

In [40]:
gallery_global    = torch.cat(gallery_global, 0) 
gallery_local     = torch.cat(gallery_local, 0) 
gallery_mask      = torch.cat(gallery_mask, 0) 
gallery_scales    = torch.cat(gallery_scales, 0) 
gallery_positions = torch.cat(gallery_positions, 0)

In [41]:
device = next(model.parameters()).device 
query_global    = query_global.to(device) 
query_local     = query_local.to(device) 
query_mask      = query_mask.to(device) 
query_scales    = query_scales.to(device) 
query_positions = query_positions.to(device) 
num_samples, top_k = cache_nn_inds.size() 
top_k = min(100, top_k)

In [42]:
top_k = min(100, top_k)
num_samples, top_k = cache_nn_inds.size()
medium_nn_inds = deepcopy(cache_nn_inds.cpu().data.numpy())

In [43]:
scores = []

In [46]:
number_str = str(123)
zero_filled_number = number_str.zfill(5)
zero_filled_number

'00123'

In [44]:
for i in tqdm(range(top_k)): 
   nnids = medium_nn_inds[:, i] 
   index_global    = gallery_global[nnids] 
   index_local     = gallery_local[nnids] 
   index_mask      = gallery_mask[nnids] 
   index_scales    = gallery_scales[nnids] 
   index_positions = gallery_positions[nnids] 
   current_scores = model( 
   query_global, query_local, query_mask, query_scales, query_positions, 
     index_global.to(device), 
     index_local.to(device), 
     index_mask.to(device), 
     index_scales.to(device), 
     index_positions.to(device)) 
   scores.append(current_scores.cpu().data)

  0%|          | 1/4993 [00:00<38:06,  2.18it/s]


RuntimeError: CUDA out of memory. Tried to allocate 270.00 MiB (GPU 0; 31.75 GiB total capacity; 29.90 GiB already allocated; 25.50 MiB free; 30.57 GiB reserved in total by PyTorch)